In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","8g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import json
import ast
sc= spark.sparkContext
df = sc.wholeTextFiles('/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/Rule_Job_0912.json').map(lambda x:ast.literal_eval(x[1]))\
                            .map(lambda x: json.dumps(x))

In [3]:
df = spark.read.json(df)

[ WARN] Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
from pyspark.sql.functions import lower, col
import pyspark.sql.functions as f
jobpost = df.select(df.JobID.alias('id'), df.JobTitle.alias('career'), df.knowledge,df.technologySkill)
jobpost = jobpost.withColumn('knowledge',lower(col('knowledge'))) \
                    .withColumn('technologySkill',lower(col('technologySkill'))) \
                    .withColumn("knowledge", f.regexp_replace(f.col("knowledge"), ",", "").alias("knowledge")) 

In [5]:
import pyspark.sql.functions as f
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import split, col, monotonically_increasing_id
from pyspark.sql.functions import lower, col
jobpost = jobpost.withColumn("timestamp", f.current_timestamp()).alias("timestamp")
jobpost = jobpost.select(jobpost.id, jobpost.career.alias('name'), jobpost.knowledge, jobpost.technologySkill, jobpost.timestamp)

In [6]:
from pyspark.sql.functions import trim,ltrim,rtrim
jobpost = jobpost.withColumn("name", ltrim(col("name"))).alias("name") \
                .withColumn('name', lower(col('name')))
jobpost = jobpost.withColumn("name", f.regexp_replace(f.col("name"), "senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "senior', ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr.", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), ", senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr. ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "jr.", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "mid-senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "lead", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "with", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "remote", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "onsite", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "mid Level", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "associate", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), ", consultant", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "senior Consultant", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "iii", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "ii", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " , ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " intern ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " intern", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "junior ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " junior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "campaign ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "contract: ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "\(.*?\)", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr ", "").alias("name")) \
    .withColumn("name", trim(col("name"))).alias("name")
jobpost = jobpost.withColumn("name", ltrim(col("name"))).alias("name") \
                .withColumn('name', lower(col('name')))

In [7]:
career = jobpost.dropDuplicates(['name']).select(jobpost.name)

In [28]:
career = career.dropDuplicates(['name']).select('name')
career.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Career")\
    .save()

In [8]:
jobpost_node = jobpost.select(jobpost.id, jobpost.timestamp)

In [48]:
jobpost_node.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "id")\
    .option("labels",":JobPosting")\
    .save()

In [49]:
jobpost.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'ABOUT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Career')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [9]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from transformers import BertForTokenClassification

2023-02-18 15:10:40.458068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 15:10:41.827234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-18 15:10:41.827267: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-18 15:10:45.074001: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [10]:
from sparknlp.annotator import *
MODEL_NAME = '/home/worker2/Desktop/thesis/NER/NERModel_config'
bert = BertForTokenClassification.loadSavedModel(
     '{}/converting/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

2023-02-18 15:10:50.299700: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-18 15:10:50.338782: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2023-02-18 15:10:50.338849: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-18 15:10:50.338953: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 15:10:50.602986: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:2

In [11]:
bert.write().overwrite().save("./{}".format(MODEL_NAME))

In [12]:
tokenClassifier_loaded = BertForTokenClassification.load("./{}".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

In [13]:
tokenClassifier_loaded.getClasses()

['I-TOOL',
 'B-TOOL',
 'I-KNOW',
 '[SEP]',
 'B-LANG',
 'I-LANG',
 'B-FRAM',
 'I-FRAM',
 'B-KNOW',
 'I-PLAT',
 '[CLS]',
 'O',
 'B-PLAT']

In [14]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('knowledge') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter()\
    .setInputCols(["document","token","ner"])\
    .setOutputCol("ner_span")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded,
    converter
])

In [15]:
knowledge_df = jobpost.select(jobpost.id, jobpost.knowledge)

In [16]:
result = pipeline.fit(knowledge_df).transform(knowledge_df)

In [17]:
import pyspark.sql.functions as F
result_last = result.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities"), 'id') \
.select(F.expr("entities['result']").alias("name"), 
        F.expr("entities['metadata'].entity").alias("entity"), 'id')

In [22]:
programmingLanguage = result_last.distinct().select(result_last.name).where(result_last.entity == 'LANG')
programmingLanguage.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:ProgrammingLanguage")\
    .save()

In [23]:
tool = result_last.distinct().select(result_last.name).where(result_last.entity == 'TOOL')
tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Tool")\
    .save()

In [24]:
framework = result_last.distinct().select(result_last.name).where(result_last.entity == 'FRAM')
framework.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Framework")\
    .save()

In [25]:
platform = result_last.distinct().select(result_last.name).where(result_last.entity == 'PLAT')
platform.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Platform")\
    .save()

In [26]:
knowledge = result_last.distinct().select(result_last.name).where(result_last.entity == 'KNOW')
knowledge.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Knowledge")\
    .save()

In [27]:
jobpost_knowledgedf_df = result_last.distinct().select(result_last.name, result_last.entity,result_last.id)

In [28]:
jobpost_pl = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'LANG')
jobpost_pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_PL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':ProgrammingLanguage')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [29]:
jobpost_tool = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'TOOL')
jobpost_tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_TOOL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Tool')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [30]:
jobpost_fram = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'FRAM')
jobpost_fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_FRAM')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Framework')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [31]:
jobpost_plat = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'PLAT')
jobpost_plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_PLAT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Platform')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [32]:
jobpost_know = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'KNOW')
jobpost_know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_KNOW')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Knowledge')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [33]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('tech') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter()\
    .setInputCols(["document","token","ner"])\
    .setOutputCol("ner_span")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded,
    converter
])

In [34]:
tech_df = jobpost.select(jobpost.id, jobpost.technologySkill.alias('tech'))

In [35]:
tech_df = tech_df.select(col("id"), split(col("tech"),",").alias("tech"))

In [36]:
from pyspark.sql.functions import explode
tech_df = tech_df.select(tech_df.id, explode("tech").alias("tech"))

In [37]:
tech_df = tech_df.withColumn("tech", trim(col("tech"))).alias("tech")\
                .withColumn("tech", ltrim(col("tech"))).alias("tech")

In [38]:
result = pipeline.fit(tech_df).transform(tech_df)

In [39]:
result_last = result.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities"), 'id') \
.select(F.expr("entities['result']").alias("name"), 
        F.expr("entities['metadata'].entity").alias("entity"), 'id')

In [40]:
result_last = result_last.orderBy("id").where(result_last.entity != 'EP]')

In [41]:
programmingLanguage = result_last.distinct().select(result_last.name).where(result_last.entity == 'LANG')
programmingLanguage.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:ProgrammingLanguage")\
    .save()

In [42]:
tool = result_last.distinct().select(result_last.name).where(result_last.entity == 'TOOL')
tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Tool")\
    .save()

In [43]:
framework = result_last.distinct().select(result_last.name).where(result_last.entity == 'FRAM')
framework.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Framework")\
    .save()

In [44]:
platform = result_last.distinct().select(result_last.name).where(result_last.entity == 'PLAT')
platform.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Platform")\
    .save()

In [45]:
knowledge = result_last.distinct().select(result_last.name).where(result_last.entity == 'KNOW')
knowledge.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("node.keys", "name")\
    .option("labels",":Competency:Knowledge")\
    .save()

In [46]:
jobpost_knowledgedf_df = result_last.distinct().select(result_last.name, result_last.entity,result_last.id)

In [47]:
jobpost_pl = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'LANG')
jobpost_pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_PL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':ProgrammingLanguage')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [48]:
jobpost_tool = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'TOOL')
jobpost_tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_TOOL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Tool')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [49]:
jobpost_fram = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'FRAM')
jobpost_fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_FRAM')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Framework')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [50]:
jobpost_plat = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'PLAT')
jobpost_plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_PLAT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Platform')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [51]:
jobpost_know = jobpost_knowledgedf_df.select(jobpost_knowledgedf_df.name, jobpost_knowledgedf_df.id).where(jobpost_knowledgedf_df.entity == 'KNOW')
jobpost_know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j://localhost:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "password")\
    .option("relationship", 'NEED_KNOW')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':JobPosting')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Knowledge')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()